In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations

In [77]:
Data = pd.read_csv('./processed data/dataframe_W256_S64_DWTLmax_samelabel_mc.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'G07_Freezing_Trial1_trial_1_emg.csv'
drop2= 'P812_M050_2_B_FoG_trial_2_emg.csv'
drop3= 'P812_M050_2_B_FoG_trial_1_emg.csv'
ind_drop = (df.columns!=drop)# & (df.columns!=drop2) & (df.columns!=drop3)
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns),np.array(df3.loc[:,0])])
ind = Data.File.isin(df.columns)
ind2 = Data.File == drop
#ind = (Data.File != drop) & (Data.File != drop2)
Data_sel = Data[ind]
Data_rest = Data[ind2]
#ind2 = Data_rest.File == drop
#Data_rest = Data_rest[ind2]
#Data_rest

In [88]:
for i in df.columns:
    print(i)

G04_FoG_trial_1_emg.csv
G04_FoG_trial_2_emg.csv
G06_FoG_trial_1_emg.csv
G06_FoG_trial_2_emg.csv
G06_FoG_trial_3_emg.csv
G07_Freezing_Trial1_trial_1_emg.csv
G08_FoG_1_trial_1_emg.csv
G08_FoG_2_trial_1_emg.csv
G11_FoG_trial_1_emg.csv
G11_FoG_trial_2_emg.csv
P379_M050_2_OFF_A_FoG_trial_1_emg.csv
P379_M050_2_OFF_A_FoG_trial_2_emg.csv
P379_M050_2_OFF_A_FoG_trial_3_emg.csv
P379_M050_2_OFF_B_FoG_trial_1_emg.csv
P379_M050_2_OFF_B_FoG_trial_2_emg.csv
P379_M050_2_OFF_B_FoG_trial_3_emg.csv
P551_M050_2_A_FoG_trial_1_emg.csv
P551_M050_2_B_FoG_trial_1_emg.csv
P551_M050_2_B_FoG_trial_2_emg.csv
P812_M050_2_B_FoG_trial_1_emg.csv
P812_M050_2_B_FoG_trial_2_emg.csv


In [90]:
feature = pd.DataFrame()
feature2 = pd.DataFrame()
y_ann = pd.DataFrame()
y2_ann = pd.DataFrame()
m = 0
for i in df.columns:
    print(i)
    ind = Data.File==i
    temp = Data[ind].reset_index(drop=True)
    ind3 = []
    ind4 = []
    for j in set(temp.Label):
        #ind2 = temp.Label == j
        #temp2 = temp[ind2]
        #l = len(temp2)
        #feature = pd.concat([feature,temp2.iloc[:int(0.8*l),1:-1]])
        #y_ann = pd.concat([y_ann,temp2.iloc[:int(0.8*l),0]])
        #feature2 = pd.concat([feature2,temp2.iloc[int(0.8*l):,1:-1]])
        #y2_ann = pd.concat([y2_ann,temp2.iloc[int(0.8*l):,0]])
        ind2 = np.where(temp.Label == j)[0].tolist()
        l_t = len(ind2)
        ind3 += ind2[:int(l_t*0.8)]
        ind4 += ind2[int(l_t*0.8):] 
    feature = pd.concat([feature,temp.iloc[ind3,1:-1]])
    y_ann = pd.concat([y_ann,temp.iloc[ind3,0]])
    feature2 = pd.concat([feature2,temp.iloc[ind4,1:-1]])
    y2_ann = pd.concat([y2_ann,temp.iloc[ind4,0]])
y_ann = np.array(y_ann)[:,0]
y2_ann = np.array(y2_ann)[:,0]

G04_FoG_trial_1_emg.csv
G04_FoG_trial_2_emg.csv
G06_FoG_trial_1_emg.csv
G06_FoG_trial_2_emg.csv
G06_FoG_trial_3_emg.csv
G07_Freezing_Trial1_trial_1_emg.csv
G08_FoG_1_trial_1_emg.csv
G08_FoG_2_trial_1_emg.csv
G11_FoG_trial_1_emg.csv
G11_FoG_trial_2_emg.csv
P379_M050_2_OFF_A_FoG_trial_1_emg.csv
P379_M050_2_OFF_A_FoG_trial_2_emg.csv
P379_M050_2_OFF_A_FoG_trial_3_emg.csv
P379_M050_2_OFF_B_FoG_trial_1_emg.csv
P379_M050_2_OFF_B_FoG_trial_2_emg.csv
P379_M050_2_OFF_B_FoG_trial_3_emg.csv
P551_M050_2_A_FoG_trial_1_emg.csv
P551_M050_2_B_FoG_trial_1_emg.csv
P551_M050_2_B_FoG_trial_2_emg.csv
P812_M050_2_B_FoG_trial_1_emg.csv
P812_M050_2_B_FoG_trial_2_emg.csv


In [5]:
import tensorflow as tf
from tensorflow.keras import layers,Model,callbacks,regularizers,models
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split,cross_validate
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder,normalize,MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE

In [31]:
#ind = ((y[:20000]==1) | (y[:20000]==2) | (y[:20000]==3) | (y[:20000]==4) | (y[:20000]==6))
#y_02 = y[:20000]
#ind = ((y==1) | (y==2) | (y==3) | (y==4) | (y==6))
file = './processed data/cwt_W256_S64_Wc32_mexh_split.hdf5'
with h5py.File(file,'r') as f:
    cwtmatr = f['features']
    y = f['labels'][...]
    cwtmatr2 = f['features2']
    y2 = f['labels2'][...]
    ind1 = ((y==1) | (y==2) | (y==6))
    ind = np.where(ind1==True)
    #print(ind)
    oc = OneHotEncoder()
    #y_02 = y.copy()
    y_02 = oc.fit_transform(y[ind1,np.newaxis]).toarray()
    #y_02[ind]=1
    ind2 = ((y2==1) | (y2==2) | (y2==6))
    ind22 = np.where(ind2==True)
    X2 = cwtmatr2[ind22]
    Y2 = oc.transform(y2[ind2,np.newaxis]).toarray()
    X_full,X_test,y_full,y_test = train_test_split(cwtmatr[ind],y_02,test_size = 0.2,shuffle = True,random_state=123)
    #X_full,X_test,y_full,y_test= train_test_split(X_full1,y_full1,test_size = 0.2,shuffle = True,random_state=123)
    #X_train,X_valid,y_train,y_valid= train_test_split(X_full,y_full,test_size = 0.2,random_state=555)

In [93]:
y_ann

array([0., 0., 0., ..., 3., 3., 3.])

In [9]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==6))
        #ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y[ind].copy()
        ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-2')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    sc = StandardScaler(with_mean=True)
    #sc = MinMaxScaler()
    x_train = sc.fit_transform(x_full)
    pca = PCA(n_components=100)
    #x_train = pca.fit_transform(x_full)
    #x_valid = sc.transform(x_valid)
    x_test = sc.transform(x_test)
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_loss', 
                                             restore_best_weights=True)
    history = model.fit([x_train,X_full],y_full,validation_data=[[x_test,X_test],y_test],
                        epochs=300,batch_size=32,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict([x_test,X_test])
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict([x_train,X_full])
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict([x_test,X_test])
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict([x_train,X_full])
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc,pca

def test_model(model,feature,y,sc,pca,binary=True):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        #ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y[ind].copy()
        ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}

    #print(y_01)
    feature=sc.transform(feature[ind])
    #feature = feature[ind]
    #feature=pca.transform(feature)
    y_pred=model.predict([feature,X2])
    test = metrics.accuracy_score(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1))
    #test = metrics.accuracy_score(y_01,y_pred>0.5)
    
    print('acc:%f'%test)
    print(metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1)))
    #print(metrics.confusion_matrix(y_01,y_pred>0.5))
    return test

def sparse_cost_sensitive_loss (y_true,y_pred):
    #cost_matrix = tf.constant([[0,1.,1,1.],
    #              [2,0,5,5],
    #              [1,1,0,1],
    #              [1.,2.,1,0]])
    cost_matrix = tf.constant([[0,2.,2],
                  [1,0,1],
                  [1.0,1.,0]])
    #cost_matrix = tf.constant([[0,1.],
    #              [5.,0]])
    batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
    eps = 1e-6
    probability = tf.clip_by_value(y_pred, eps, 1-eps)
    cost_values = tf.math.log(1-probability)*batch_cost_matrix
    loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
    return loss

In [135]:
input_ann = layers.Input(shape=feature.shape[1:])#feature.shape[1:]
l1 = layers.Dense(128,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(input_ann)
l1 = layers.Dropout(0.2)(l1)
#bn1 = layers.BatchNormalization()(l1)

l2 = layers.Dense(64,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(l1)
l2 = layers.Dropout(0.2)(l2)
#bn2 = layers.BatchNormalization()(l2)

l3 = layers.Dense(32,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(l2)
l3 = layers.Dropout(0.2)(l3)

#bn3 = layers.BatchNormalization()(l3)
l4 = layers.Dense(16,activation='elu',
                 #kernel_regularizer = regularizers.l2(0.001),
                 )(l3)
#l4 = layers.Dropout(0.2)(l4)

#l5 = layers.Dense(8,activation='relu')(drop4)
#drop5 = layers.Dropout(0.5)(l5)


input_cnn = layers.Input(shape=X_full.shape[1:])
#bn = layers.BatchNormalization()(input_)
max_pool = layers.MaxPooling2D((2,2))(input_cnn)
#bn = layers.BatchNormalization()(input_)
cnn1 = layers.Conv2D(16,(3,6),strides=(1,1),
                      kernel_initializer=TruncatedNormal(),
                      #use_bias=False,#activation='elu',
                      padding='same')(max_pool)
cnn1 = layers.Activation('elu')(cnn1)
cnn1 = layers.BatchNormalization()(cnn1)
cnn1 = layers.MaxPooling2D((2,2))(cnn1)

cnn2 = layers.Conv2D(32,(3,6),strides=(1,1),
                      kernel_initializer=TruncatedNormal(),
                      #use_bias=False,#activation='elu',
                      padding='same')(cnn1)
cnn2 = layers.Activation('elu')(cnn2)
cnn2 = layers.BatchNormalization()(cnn2)
cnn2 = layers.MaxPooling2D(2)(cnn2)

cnn3 = layers.Conv2D(64,(3,6),strides=(1,1),
                      kernel_initializer=TruncatedNormal(),
                      #use_bias=False,#activation='elu',
                      padding='same')(cnn2)
cnn3 = layers.Activation('elu')(cnn3)
cnn3 = layers.BatchNormalization()(cnn3)
cnn3 = layers.MaxPooling2D(2)(cnn3)

cnn4 = layers.Conv2D(128,(3,6),strides=(1,1),
                      kernel_initializer=TruncatedNormal(),
                      #use_bias=False,#activation='elu',
                      padding='same')(cnn3)
cnn4 = layers.Activation('elu')(cnn4)
cnn4 = layers.BatchNormalization()(cnn4)
cnn4 = layers.MaxPooling2D(2)(cnn4)
flatten = layers.Flatten()(cnn4)

#dropout = layers.Dropout(0.2)(flatten)
#dense1 = layers.Dense(256,activation = 'relu')(dropout)
#dense1 = layers.Dropout(0.2)(dense1)
#dense2 = layers.Dense(128,activation = 'relu')(dense1)
#dropout2 = layers.Dropout(0.2)(layer2)

concat = layers.Concatenate()([l4,flatten])

output = layers.Dense(3,activation='softmax')(concat)
model = Model(inputs=[input_ann,input_cnn],outputs=[output])

In [136]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [138]:
train,test,sc,pca = train_model(model,feature,np.array(y),False)

Train on 6812 samples, validate on 1704 samples
Epoch 1/300
6812/6812 [==============================] - 69s 10ms/sample - loss: 0.7756 - accuracy: 0.6723 - val_loss: 1.5976 - val_accuracy: 0.2799
Epoch 2/300
6812/6812 [==============================] - 55s 8ms/sample - loss: 0.6643 - accuracy: 0.7124 - val_loss: 0.7905 - val_accuracy: 0.6590
Epoch 3/300
6812/6812 [==============================] - 61s 9ms/sample - loss: 0.6150 - accuracy: 0.7366 - val_loss: 0.5693 - val_accuracy: 0.7576
Epoch 4/300
6812/6812 [==============================] - 55s 8ms/sample - loss: 0.5861 - accuracy: 0.7551 - val_loss: 0.5472 - val_accuracy: 0.7570
Epoch 5/300
6812/6812 [==============================] - 52s 8ms/sample - loss: 0.5560 - accuracy: 0.7672 - val_loss: 0.6229 - val_accuracy: 0.7218
Epoch 6/300
6812/6812 [==============================] - 55s 8ms/sample - loss: 0.5204 - accuracy: 0.7805 - val_loss: 0.5154 - val_accuracy: 0.7782
Epoch 7/300
6812/6812 [==============================] - 54s 8m

Epoch 56/300
6812/6812 [==============================] - 33s 5ms/sample - loss: 0.0216 - accuracy: 0.9935 - val_loss: 0.2341 - val_accuracy: 0.9331
Epoch 57/300
6812/6812 [==============================] - 34s 5ms/sample - loss: 0.0136 - accuracy: 0.9957 - val_loss: 0.2607 - val_accuracy: 0.9442
Epoch 58/300
6812/6812 [==============================] - 33s 5ms/sample - loss: 0.0128 - accuracy: 0.9962 - val_loss: 0.3618 - val_accuracy: 0.9290
Epoch 59/300
6812/6812 [==============================] - 34s 5ms/sample - loss: 0.0215 - accuracy: 0.9919 - val_loss: 0.3518 - val_accuracy: 0.9208
Epoch 60/300
6812/6812 [==============================] - 33s 5ms/sample - loss: 0.0203 - accuracy: 0.9932 - val_loss: 0.4929 - val_accuracy: 0.9008
Epoch 61/300
6812/6812 [==============================] - 33s 5ms/sample - loss: 0.0201 - accuracy: 0.9938 - val_loss: 0.3015 - val_accuracy: 0.9202
Epoch 62/300
6812/6812 [==============================] - 33s 5ms/sample - loss: 0.0215 - accuracy: 0.9938

In [139]:
test_model(model,np.array(feature2),np.array(y2),sc,pca,False)

acc:0.570229
[[278  48 150]
 [221 568 288]
 [214   0 376]]


0.5702286514232384

In [196]:
model_ann = models.load_model('./model/ann.h5')
model_cnn = models.load_model('./model/cnn.h5')

In [227]:
input_ann = layers.Input(shape=feature.shape[1:])
ann = model_ann.layers[0](input_ann)
ann.trainable = False
for layer in model_ann.layers[1:-2]:
    ann = layer(ann)
    ann.trainable = False
#model_ann = Model(inputs=[input_],outputs=[ann])

input_cnn = layers.Input(shape=X_full.shape[1:])
cnn = model_cnn.layers[0](input_cnn)
cnn .trainable = False
for layer in model_cnn.layers[2:-1]:
    cnn = layer(cnn)
    cnn.trainable = False
#model_cnn = Model(inputs=[input_cnn],outputs=[cnn])
concat = layers.Concatenate()([ann,cnn])
dense1 = layers.Dense(1024,activation='relu')(concat)
dense1 = layers.Dropout(0.2)(dense1)
dense2 = layers.Dense(512,activation='relu')(dense1)
dense2 = layers.Dropout(0.2)(dense2)
dense3 = layers.Dense(256,activation='relu')(dense2)
dense3 = layers.Dropout(0.2)(dense3)
output = layers.Dense(3,activation='softmax')(dense3)

model = Model(inputs=[input_ann,input_cnn],outputs=[output])
for layer in model.layers[:-6]:
    layer.trainable=False
model.summary()

Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_76 (InputLayer)           [(None, 32, 256, 8)] 0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           multiple             0           input_76[0][0]                   
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              multiple             1552        input_27[7][0]                   
__________________________________________________________________________________________________
activation_56 (Activation)      multiple             0           conv2d_56[6][0]                  
___________________________________________________________________________________________

In [222]:
model.layers[2].trainable=False
model.layers[-5]

In [228]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [229]:
train,test,sc,pca = train_model(model,feature,np.array(y),False)

Train on 6812 samples, validate on 1704 samples
Epoch 1/300
6812/6812 [==============================] - 61s 9ms/sample - loss: 0.1799 - accuracy: 0.9306 - val_loss: 0.2565 - val_accuracy: 0.9096
Epoch 2/300
6812/6812 [==============================] - 67s 10ms/sample - loss: 0.0415 - accuracy: 0.9866 - val_loss: 0.2906 - val_accuracy: 0.9243
Epoch 3/300
6812/6812 [==============================] - 60s 9ms/sample - loss: 0.0313 - accuracy: 0.9894 - val_loss: 0.3368 - val_accuracy: 0.9173
Epoch 4/300
6812/6812 [==============================] - 59s 9ms/sample - loss: 0.0298 - accuracy: 0.9919 - val_loss: 0.3970 - val_accuracy: 0.9049
Epoch 5/300
6812/6812 [==============================] - 59s 9ms/sample - loss: 0.0207 - accuracy: 0.9930 - val_loss: 0.3764 - val_accuracy: 0.9137
Epoch 6/300
6812/6812 [==============================] - 59s 9ms/sample - loss: 0.0155 - accuracy: 0.9952 - val_loss: 0.4694 - val_accuracy: 0.9114
Epoch 7/300
6812/6812 [==============================] - 65s 10

In [226]:
test_model(model,np.array(feature2),np.array(y2),sc,pca,False)

acc:0.737284
[[150 216 110]
 [ 66 964  47]
 [ 44  80 466]]


0.7372841810545964

In [148]:
input_ = layers.Input(shape=X_full.shape[1:])
#bn = layers.BatchNormalization()(input_)
max_pool = layers.MaxPooling2D((2,2))(input_)
#bn = layers.BatchNormalization()(input_)

cnn1 = layers.Conv2D(16,(2,6),strides=(1,1),
                      kernel_initializer=TruncatedNormal(),
                      #use_bias=False,#activation='elu',
                      padding='same')(max_pool)
cnn1 = layers.Activation('relu')(cnn1)
cnn1 = layers.BatchNormalization()(cnn1)
cnn1 = layers.MaxPooling2D((2,2))(cnn1)

cnn2 = layers.Conv2D(32,(2,6),strides=(1,1),
                      kernel_initializer=TruncatedNormal(),
                      #use_bias=False,#activation='elu',
                      padding='same')(cnn1)
cnn2 = layers.Activation('relu')(cnn2)
cnn2 = layers.BatchNormalization()(cnn2)
cnn2 = layers.MaxPooling2D(2)(cnn2)

cnn3 = layers.Conv2D(64,(2,6),strides=(1,1),
                      kernel_initializer=TruncatedNormal(),
                      #use_bias=False,#activation='elu',
                      padding='same')(cnn2)
cnn3 = layers.Activation('relu')(cnn3)
cnn3 = layers.BatchNormalization()(cnn3)
cnn3 = layers.MaxPooling2D(2)(cnn3)

cnn4 = layers.Conv2D(128,(2,6),strides=(1,1),
                      kernel_initializer=TruncatedNormal(),
                      #use_bias=False,#activation='elu',
                      padding='same')(cnn3)
cnn4 = layers.Activation('relu')(cnn4)
cnn4 = layers.BatchNormalization()(cnn4)
cnn4 = layers.MaxPooling2D(2)(cnn4)

flatten = layers.Flatten()(cnn4)
#dropout = layers.Dropout(0.2)(flatten)
#layer1 = layers.Dense(256,activation = 'relu')(dropout)
#dropout1 = layers.Dropout(0.2)(layer1)
#layer2 = layers.Dense(128,activation = 'relu')(dropout1)
#dropout2 = layers.Dropout(0.2)(layer2)
output = layers.Dense(3,activation = 'softmax')(flatten)
model = Model(inputs=[input_],outputs=[output])

In [149]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [145]:
early_stopping = callbacks.EarlyStopping(patience = 10,monitor = 'val_accuracy', restore_best_weights=True)

In [150]:
history = model.fit(X_full,y_full,validation_data=[X_test,y_test],
                    epochs=300,batch_size=32,
                    #class_weight=cw,
                    #callbacks=[early_stopping]
                   )

Train on 6812 samples, validate on 1704 samples
Epoch 1/300
6812/6812 [==============================] - 38s 6ms/sample - loss: 0.8749 - accuracy: 0.6735 - val_loss: 1.5026 - val_accuracy: 0.2758
Epoch 2/300
6812/6812 [==============================] - 35s 5ms/sample - loss: 0.6004 - accuracy: 0.7449 - val_loss: 0.8417 - val_accuracy: 0.5804
Epoch 3/300
6812/6812 [==============================] - 35s 5ms/sample - loss: 0.5318 - accuracy: 0.7689 - val_loss: 0.5847 - val_accuracy: 0.7465
Epoch 4/300
6812/6812 [==============================] - 36s 5ms/sample - loss: 0.4653 - accuracy: 0.8077 - val_loss: 0.5940 - val_accuracy: 0.7600
Epoch 5/300
6812/6812 [==============================] - 37s 5ms/sample - loss: 0.3698 - accuracy: 0.8523 - val_loss: 0.5277 - val_accuracy: 0.7964
Epoch 6/300
6812/6812 [==============================] - 44s 7ms/sample - loss: 0.2748 - accuracy: 0.8920 - val_loss: 0.4538 - val_accuracy: 0.8281
Epoch 7/300
6812/6812 [==============================] - 37s 5ms

Epoch 56/300
6812/6812 [==============================] - 30s 4ms/sample - loss: 0.0063 - accuracy: 0.9977 - val_loss: 0.2073 - val_accuracy: 0.9513
Epoch 57/300
6812/6812 [==============================] - 29s 4ms/sample - loss: 0.0037 - accuracy: 0.9987 - val_loss: 0.1399 - val_accuracy: 0.9630
Epoch 58/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0028 - accuracy: 0.9991 - val_loss: 0.1471 - val_accuracy: 0.9630
Epoch 59/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.1613 - val_accuracy: 0.9601
Epoch 60/300
6812/6812 [==============================] - 29s 4ms/sample - loss: 0.0133 - accuracy: 0.9949 - val_loss: 0.3807 - val_accuracy: 0.9161
Epoch 61/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0459 - accuracy: 0.9859 - val_loss: 0.5706 - val_accuracy: 0.8920
Epoch 62/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0577 - accuracy: 0.9830

6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.2499 - val_accuracy: 0.9554
Epoch 165/300
6812/6812 [==============================] - 29s 4ms/sample - loss: 0.0122 - accuracy: 0.9960 - val_loss: 0.2575 - val_accuracy: 0.9454
Epoch 166/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0096 - accuracy: 0.9974 - val_loss: 0.3412 - val_accuracy: 0.9431
Epoch 167/300
6812/6812 [==============================] - 33s 5ms/sample - loss: 0.0086 - accuracy: 0.9971 - val_loss: 0.2587 - val_accuracy: 0.9466
Epoch 168/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0064 - accuracy: 0.9984 - val_loss: 0.2469 - val_accuracy: 0.9607
Epoch 169/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0067 - accuracy: 0.9978 - val_loss: 0.3411 - val_accuracy: 0.9442
Epoch 170/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0063 - accuracy: 0.9982 - val_

6812/6812 [==============================] - 29s 4ms/sample - loss: 1.3996e-04 - accuracy: 1.0000 - val_loss: 0.2263 - val_accuracy: 0.9554
Epoch 273/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 1.0789e-04 - accuracy: 1.0000 - val_loss: 0.2298 - val_accuracy: 0.9572
Epoch 274/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 9.0233e-05 - accuracy: 1.0000 - val_loss: 0.2066 - val_accuracy: 0.9619
Epoch 275/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 1.2084e-04 - accuracy: 1.0000 - val_loss: 0.2098 - val_accuracy: 0.9613
Epoch 276/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 3.3861e-05 - accuracy: 1.0000 - val_loss: 0.1963 - val_accuracy: 0.9636
Epoch 277/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 1.5473e-04 - accuracy: 1.0000 - val_loss: 0.2120 - val_accuracy: 0.9613
Epoch 278/300
6812/6812 [==============================] - 28s 4ms/sample - loss: 0.0012 -

In [153]:
model.evaluate(X2,Y2)

2143/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[3.359199850518766, 0.7116192]

In [152]:
y_pred=model.predict(X2)
metrics.confusion_matrix(np.argmax(Y2,axis=1),np.argmax(y_pred,axis=1))

array([[191, 184, 101],
       [121, 898,  58],
       [ 56,  98, 436]], dtype=int64)

In [166]:
y_pred_t=model.predict(X_test)
test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
#y_pred_v=model.predict(x_valid)
#valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
y_pred_ta=model.predict(X_full)
train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
print('test:%f'%test)
#print('valid:%f'%valid)
print('train:%f'%train)
print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
#print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))

test:0.966549
train:1.000000
train: 
 [[1503    0    0]
 [   0 3425    0]
 [   0    0 1884]]
test: 
 [[371  10  10]
 [ 15 823   8]
 [  9   5 453]]


In [154]:
model.save('./model/cnn.h5')